# Computing in Context: Public Policy
## Project 2: GroupBy 

---

In this project, you will use what you have learned about `GroupBy` operations in Pandas to answer questions about a dataset as well as construct new datasets at different units of analysis. This is a common data task in policy analysis. You might receive a dataset at a person-level and need to create household-level data instead, or receive data at a household-level and need to create geographic-level data.

You will be working with the [DOHMH New York City Restaurant Inspection Results](https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j) dataset, made available on the [NYC Open Data](http://opendata.cityofnewyork.us/) portal. Specifically, you will be working with a sample of the data in the file `restaurant_inspection_results.csv` that I have posted to Courseworks.

To submit the project, upload the completed notebook to Courseworks.

This is an individual assignment.

In [1]:
# Import relevant libraries
import pandas as pd
import matplotlib.pyplot as plt

# Make sure notebook prints plots
%matplotlib inline

---

### Task 1

Load the data, including the following columns:

- `CAMIS`
- `DBA`
- `BORO`
- `CUISINE DESCRIPTION`
- `INSPECTION DATE`
- `VIOLATION CODE`
- `VIOLATION DESCRIPTION`
- `CRITICAL FLAG`
- `SCORE`
- `Census Tract`

Make sure to load any date columns as date type.

In [2]:
df = pd.read_csv('restaurant_inspection_results.csv')
df[['INSPECTION DATE','GRADE DATE','RECORD DATE']] = df[['INSPECTION DATE','GRADE DATE','RECORD DATE']].apply(pd.to_datetime)

---

### Task 2

Use `groupby` to ask and answer a question of this data that we didn't cover in class.

In [3]:
## Checking which day of the week maximum ispections are done
df['inspection_month'] = df['INSPECTION DATE'].dt.month
df['inspection_dayofweek'] = df['INSPECTION DATE'].dt.weekday_name
df.groupby('inspection_dayofweek')['CAMIS'].count().sort_values()

inspection_dayofweek
Sunday          7
Saturday      473
Friday       2002
Monday       2556
Tuesday      2668
Wednesday    2881
Thursday     3765
Name: CAMIS, dtype: int64

---

### Task 3

What does each row of the current dataset represent? If you wanted to create a restaurant-level dataset (where each row in the dataframe represents a restaurant), what column should you split on?

**The original dataset is at inspection/violations level, i.e a single restaurant can have multiple records if it has multiple unique violations.
To creat a restaurant-level dataset only we'll have to split at DBA column. Also if the same restaurant has had diffent names then we can use a columns like "CAMIS" and the address (and phone number!) to ensure if restraunts have changed their names

---

### Task 4

In the next two tasks, you are going to create a restaurant-level dataset where each row in the new dataframe represents a restaurant. Your restaurant-level datset should have the following features:

- The restaurant's name at the date of the most recent inspection
- The number of names associated with the restaurant
- The restaurant's cuisine description at the date of the most recent inspection
- The number of inspections the restaurant has had
- The date of the first inspection
- The date of the most recent inspection
- The number of  inspection citations the restaurant has had
- The number of unique violation types of the restaurant has had
- Whether the restaurant has had a critical violation
- The restaurant's most recent score
- The borough the restaurant is located in at the date of the most recent inspection
- The Census tract the restaurant is located in at the date of the most recent inspection

For task 3, write a function `get_restaurant_features()` that takes a restaurant group `x` from the original dataframe and returns a Series with the above features for that restaurant group.

Hints:

- Some features require you to aggregate across all data for a restaurant group. Others require you to look at data associated with the most recent inspection. How can you use what you learned about `loc` and `idxmax()` to help with the latter set of features.
- Think about the Series this function returns as a row in your final dataframe. Each value in the Series will be a particular value for a column in your restaurant dataframe, and its index should correspond to the column names you want in your restaurant dataframe.

In [4]:
def critical_violations(row):
    if 'Y' in list(row): #converting series to list
        return 'Y'
    else:
        return 'N'



#NOTE: instead of len(set(row)) I could have also used Series functions called "nunique() / unique()". I chose to directly create df as it was more efficient.   
def get_restaurant_features(x):
    df['latest_rest_name'] = x['DBA'].transform(lambda row: list(row)[0])
    df['number_of_names'] = x['DBA'].transform(lambda row: len(set(row)))
    df['latest_rest_cusin_descrip'] = x['CUISINE DESCRIPTION'].transform(lambda row: list(row)[0])
    df['num_of_inspections'] = x['INSPECTION DATE'].transform(lambda row: len(set(row)))
    df['first_inspection_date'] = x['INSPECTION DATE'].transform(lambda row: list(row)[-1])
    df['latest_inspection_date'] = x['INSPECTION DATE'].transform(lambda row: list(row)[0])
    df['number_of_citations'] = x['VIOLATION DESCRIPTION'].transform(lambda row: len(row))
    df['num_of_violation_types'] = x['VIOLATION CODE'].transform(lambda row: len(set(row)))
    df['critical_violations'] = x['CRITICAL FLAG'].transform(critical_violations)
    df['most_recent_score'] = x['SCORE'].transform(lambda row: list(row)[0])
    df['most_recent_borough'] = x['BORO'].transform(lambda row: list(row)[0])
    df['most_recent_census'] = x['Census Tract'].transform(lambda row: list(row)[0])

---

### Task 5

Create a restaurant-level dataframe using your `get_restaurant_features()` function from Task 3 and what you have learned about `GroupBy` operations.

In [5]:
x = df.sort_values(by='INSPECTION DATE',ascending =False).groupby('CAMIS')
get_restaurant_features(x)  ## Since I am using transform above, all the new columns get added to the original dataframe. Here get_restaurant_features(x) returns a dataframe

In [6]:
df[['CAMIS','latest_rest_name','number_of_names','latest_rest_cusin_descrip','num_of_inspections','first_inspection_date','latest_inspection_date','number_of_citations','num_of_violation_types','critical_violations','most_recent_score','most_recent_borough','most_recent_census']].head(20)

,CAMIS,latest_rest_name,number_of_names,latest_rest_cusin_descrip,num_of_inspections,first_inspection_date,latest_inspection_date,number_of_citations,num_of_violation_types,critical_violations,most_recent_score,most_recent_borough,most_recent_census
0,50094291,TAIYAKI NYC,1,"Ice Cream, Gelato, Yogurt, Ices",2,2019-08-01,2019-08-19,5,5,Y,7.0,Queens,87100.0
1,50056766,MYRTLE & CO,1,American,1,2019-08-01,2019-08-01,6,6,Y,54.0,Brooklyn,19100.0
2,41431239,QUE SABROSURA RESTAURANT,1,Spanish,1,2019-08-01,2019-08-01,7,7,Y,40.0,Bronx,23900.0
3,50036928,CITYLINE BAKERY,1,Bakery,1,2019-08-01,2019-08-01,3,3,Y,10.0,Brooklyn,119000.0
4,40396496,MR. PHIL'S PIZZA,1,Pizza,1,2019-08-01,2019-08-01,2,2,Y,12.0,Brooklyn,19000.0
5,40365355,NEW CORNER,1,Italian,2,2019-08-01,2019-08-27,6,6,Y,12.0,Brooklyn,13200.0
6,41624716,CLOISTERS GRILL,1,"Latin (Cuban, Dominican, Puerto Rican, South &...",1,2019-08-01,2019-08-01,8,8,Y,52.0,Manhattan,28700.0
7,50079732,CAMP DAVID,1,American,2,2019-08-01,2019-08-13,8,8,Y,12.0,Brooklyn,1800.0
8,50065974,DOMINO'S,1,Pizza,1,2019-08-01,2019-08-01,4,4,Y,13.0,Manhattan,7200.0
9,40365938,NANCY'S RESTAURANT,1,American,1,2019-08-01,2019-08-01,3,3,N,6.0,Queens,157903.0


---

### Task 6

What is one idea for a dataset you could combine with the restaurant-level dataframe? What is a question that you could answer by combining these two datasets?

I can use restaurant review dataset and combine with the inspection dataset to see if the restaurant ratings imporove. The review dataset should have atleast the following columns:

Reviewer ID/Name; Date of Review; 
CAMIS ID; 
DBA; 
Review; 
Reviewer Star Rating of the restaurant; 
Star rating of the restaurant on the day of the review. 

What I would like to see is a change in the reviews and star ratings post inspection. 